# Cosmic Shear Forecast with Fisher and DALI (PyCCL + DerivKit)

In this notebook we build a simple cosmic shear forecasting setup using **PyCCL** for the theory predictions and **DerivKit** for derivative-based likelihood expansions.

We:

* Construct tomographic source bins from a Smail redshift distribution.
* Compute cosmic shear $C_\ell^{ij}$ including:

  * Intrinsic Alignments (IA),
  * Baryonic feedback using the van Daalen (2019) model.
* Build a simple diagonal covariance model.
* Compute:

  * The **Fisher matrix** (Gaussian / quadratic approximation),
  * The **DALI expansion** (includes higher-order curvature terms).
* Compare posterior contours under identical Gaussian priors.

The goal is to illustrate how simple it is to use DerivKit to compute
likelihoods and posterior samples for forecasts.


In [ ]:
# you must run before importing numpy/pyccl/etc
# this is a problem with ccl and not with derivkit
import os

os.environ["OMP_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["VECLIB_MAXIMUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"

# Optional: make it obvious if someone re-runs later after imports
print("Thread limits set. Now import numpy/pyccl.")


In [ ]:
import numpy as np
import pyccl as ccl
import matplotlib.pyplot as plt
from getdist import plots as getdist_plots
import cmasher as cmr

from derivkit import ForecastKit

In [ ]:
def smail_source_bins(z, n_source, *, z0=0.13, alpha=0.78, beta=2.0):
    """Quick smail nz and soutce bins"""
    z = np.asarray(z, float)
    nz = (z / z0) ** beta * np.exp(-((z / z0) ** alpha))
    nz /= np.trapezoid(nz, z)

    cdf = np.concatenate([
        [0.0],
        np.cumulative_sum(0.5 * (nz[1:] + nz[:-1]) * (z[1:] - z[:-1]))
    ])

    def slice_bins(edges):
        out = []
        for lo, hi in zip(edges[:-1], edges[1:]):
            m = (z >= lo) & (z < hi if hi < edges[-1] else z <= hi)
            ni = np.where(m, nz, 0.0)
            ni /= np.trapezoid(ni, z)
            out.append(ni)
        return out

    edges_s = np.interp(np.linspace(0.0, cdf[-1], n_source + 1), cdf, z)
    edges_s[0], edges_s[-1] = z[0], z[-1]
    source_bins = slice_bins(edges_s)
    return nz, source_bins


def shear_power_spectra(
    theta: np.ndarray,
    *,
    z: np.ndarray,
    ell: np.ndarray,
    source_bins: list[np.ndarray],
) -> np.ndarray:
    """Cosmic shear Cls with IA + van Daalen19 baryon boost."""
    # these are our model parameters
    om_m, sig8, ia_amp, ia_eta, fbar = map(float, theta)

    cosmo = ccl.Cosmology(
        Omega_c=om_m - 0.045,
        Omega_b=0.045,
        h=0.67,
        sigma8=sig8,
        n_s=0.96,
        transfer_function="boltzmann_camb",
    )

    # IA: simple demo scaling (same for all source bins)
    z_p = 0.62
    ia_signal = (
        ia_amp
        * ((1.0 + z) / (1.0 + z_p)) ** ia_eta
    )

    # Build baryon-boosted nonlinear P(k,a) and pass it to angular_cl
    vd = ccl.baryons.BaryonsvanDaalen19(fbar=fbar, mass_def="500c")
    pk_nl = cosmo.get_nonlin_power()  # returns Pk2D
    pk_bar = vd.include_baryonic_effects(cosmo, pk_nl)  # returns Pk2D

    tracers = [
        ccl.WeakLensingTracer(cosmo, dndz=(z, nz_i), ia_bias=(z, ia_signal))
        for nz_i in source_bins
    ]

    n = len(tracers)
    out = np.empty((n * (n + 1) // 2) * ell.size)

    k = 0
    for a in range(n):
        for b in range(a, n):
            out[k : k + ell.size] = ccl.angular_cl(
                cosmo, tracers[a], tracers[b], ell, p_of_k_a=pk_bar
            )
            k += ell.size

    return out


In [ ]:
# setup grids + bins
ell = np.geomspace(20, 2000, 20)
z = np.linspace(0.0, 3.0, 300)

n_source = 5
nz_parent, source_bins = smail_source_bins(z, n_source=n_source)

# Omegam, sigma8, ia_amp, ia_eta, fbar
theta0 = [0.315, 0.80, 0.50, 2.2, 0.70]

def model(theta):
    """Model wrapper for ForecastKit.

    ForecastKit expects a callable of the form f(theta) -> 1D data vector, so we
    bind the fixed inputs (z, ell, source_bins) here and expose only theta.
    Of course, you can do this differently in your own pipelines.
    """
    return shear_power_spectra(theta, z=z, ell=ell, source_bins=source_bins)

y0 = model(theta0)

floor = 1e-12 * np.max(np.abs(y0))  # avoid zero
sigma_i = 0.05 * np.maximum(np.abs(y0), floor)

cov = np.diag(sigma_i**2)

fk = ForecastKit(function=model, theta0=theta0, cov=cov)

# you can use differnt derivative backends
# the default is adaptive polyfit
# note that the choice of derivative method will impact the computing time
# and accuracy of the posterior samples. Best would be to start with local
# polyfit but for the sake of this demo we use finite differences
fisher = fk.fisher(
    method="finite_diff"
)

# we do second ordder DALI which means you get DALI doublet
# tensors (see the docs for more details on what this means)
dali = fk.dali(
    forecast_order=2,
    method="finite_diff",
)


In [ ]:
names  = ["om_m", "sig8", "ia_amp", "ia_eta", "f_bar"]
labels = [r"\Omega_m",
          r"\sigma_8",
          r"A_{\rm IA}",
          r"\eta_{\rm IA}",
          r"f_{\rm bar}"]

# 10% Gaussian priors (1-sigma)
sigma_prior = 0.10 * np.abs(theta0)

fisher_prior = np.diag(1.0 / sigma_prior**2)
fisher_post = fisher + fisher_prior

fisher_samples_post = fk.getdist_fisher_gaussian(
    fisher=fisher_post,
    names=names,
    labels=labels,
    label="Fisher + 10% Gaussian prior",
)

prior_terms = [
    ("gaussian", {"mean": theta0, "cov": np.diag(sigma_prior**2)}),
]

dali_samples_post = fk.getdist_dali_emcee(
    dali=dali,
    names=names,
    labels=labels,
    label="DALI + 10% Gaussian prior",
    prior_terms=prior_terms,
)


In [ ]:
samples = [fisher_samples_post, dali_samples_post]
n = len(samples)
colors = cmr.take_cmap_colors("cmr.prinsenvlag",
                              len(samples),
                              cmap_range=(0.2, 0.8),
                              return_fmt='hex')
plotter = getdist_plots.get_subplot_plotter(width_inch=6)
plotter.triangle_plot(samples,
                      params=names,
                      filled=False,
                      contour_colors=colors,
                      contour_lws=[2] * n,
                      contour_ls=["-"] * n,
                      )
plt.show()